In [1]:
import keras,os
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
import shutil

In [2]:
DATASET_PATH  = '../../people_and_animals/resnet_data'
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 4
BATCH_SIZE = 8
FREEZE_LAYERS = 2
NUM_EPOCHS = 20
WEIGHTS_FINAL = 'resnet_nose_model.weight'

In [3]:
# 刪除 .ipynb_checlpoints
# import os
# os.rmdir("../people_and_animals/resnet_data/nose_v/.ipynb_checkpoints")

In [4]:
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/nose_train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

Found 764 images belonging to 4 classes.


In [5]:
valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/nose_v',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

Found 16 images belonging to 4 classes.


In [6]:
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Class #0 = A
Class #1 = B
Class #2 = C
Class #3 = D


In [7]:
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

In [8]:
x = Dropout(0.5)(x)

In [9]:
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

In [10]:
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

In [11]:
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
print(net_final.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [13]:
hist = net_final.fit_generator(steps_per_epoch=1,generator=train_batches, validation_data= valid_batches, validation_steps=1,epochs=1500)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/1500
1/1 [==============================] - 1s 765ms/step - loss: 2.5962 - accuracy: 0.5000 - val_loss: 2.5431 - val_accuracy: 0.1250
Epoch 2/1500
1/1 [==============================] - 0s 241ms/step - loss: 3.5701 - accuracy: 0.3750 - val_loss: 2.4849 - val_accuracy: 0.1250
Epoch 3/1500
1/1 [==============================] - 0s 241ms/step - loss: 2.4571 - accuracy: 0.5000 - val_loss: 2.4793 - val_accuracy: 0.1250
Epoch 4/1500
1/1 [==============================] - 0s 240ms/step - loss: 4.3798 - accuracy: 0.1250 - val_loss: 2.5016 - val_accuracy: 0.1250
Epoch 5/1500
1/1 [==============================] - 0s 242ms/step - loss: 2.1575 - accuracy: 0.5000 - val_loss: 2.5069 - val_accuracy: 0.1250
Epoch 6/1500
1/1 [==============================] - 0s 241ms/step - loss: 1.2293 - accuracy: 0.6250 - val_loss: 2.5163 - val_accuracy: 0.1250
Epoch 7/1500
1/1 [==============================] - 0s 260ms/step - loss

1/1 [==============================] - 0s 241ms/step - loss: 4.7483 - accuracy: 0.2500 - val_loss: 1.4883 - val_accuracy: 0.5000
Epoch 57/1500
1/1 [==============================] - 0s 246ms/step - loss: 1.2589 - accuracy: 0.6250 - val_loss: 1.4760 - val_accuracy: 0.5000
Epoch 58/1500
1/1 [==============================] - 0s 244ms/step - loss: 3.8031 - accuracy: 0.1250 - val_loss: 1.4643 - val_accuracy: 0.5000
Epoch 59/1500
1/1 [==============================] - 0s 240ms/step - loss: 2.3319 - accuracy: 0.3750 - val_loss: 1.4687 - val_accuracy: 0.5000
Epoch 60/1500
1/1 [==============================] - 0s 238ms/step - loss: 2.1002 - accuracy: 0.5000 - val_loss: 1.4690 - val_accuracy: 0.5000
Epoch 61/1500
1/1 [==============================] - 0s 240ms/step - loss: 1.3710 - accuracy: 0.5000 - val_loss: 1.4705 - val_accuracy: 0.5000
Epoch 62/1500
1/1 [==============================] - 0s 244ms/step - loss: 1.3422 - accuracy: 0.5000 - val_loss: 1.4888 - val_accuracy: 0.5000
Epoch 63/1500

1/1 [==============================] - 0s 243ms/step - loss: 0.4991 - accuracy: 0.7500 - val_loss: 0.7152 - val_accuracy: 0.7500
Epoch 114/1500
1/1 [==============================] - 0s 246ms/step - loss: 1.4337 - accuracy: 0.3750 - val_loss: 0.6889 - val_accuracy: 0.7500
Epoch 115/1500
1/1 [==============================] - 0s 242ms/step - loss: 2.3896 - accuracy: 0.3750 - val_loss: 0.6652 - val_accuracy: 0.7500
Epoch 116/1500
1/1 [==============================] - 0s 245ms/step - loss: 2.9526 - accuracy: 0.2500 - val_loss: 0.6463 - val_accuracy: 0.7500
Epoch 117/1500
1/1 [==============================] - 0s 242ms/step - loss: 1.8903 - accuracy: 0.3750 - val_loss: 0.6310 - val_accuracy: 0.7500
Epoch 118/1500
1/1 [==============================] - 0s 245ms/step - loss: 2.2997 - accuracy: 0.3750 - val_loss: 0.6187 - val_accuracy: 0.7500
Epoch 119/1500
1/1 [==============================] - 0s 247ms/step - loss: 1.8851 - accuracy: 0.6250 - val_loss: 0.6017 - val_accuracy: 0.7500
Epoch 1

Epoch 170/1500
1/1 [==============================] - 0s 243ms/step - loss: 1.0832 - accuracy: 0.7500 - val_loss: 0.6712 - val_accuracy: 0.7500
Epoch 171/1500
1/1 [==============================] - 0s 241ms/step - loss: 3.0122 - accuracy: 0.3750 - val_loss: 0.6746 - val_accuracy: 0.7500
Epoch 172/1500
1/1 [==============================] - 0s 242ms/step - loss: 0.9615 - accuracy: 0.7500 - val_loss: 0.6751 - val_accuracy: 0.7500
Epoch 173/1500
1/1 [==============================] - 0s 246ms/step - loss: 1.2753 - accuracy: 0.5000 - val_loss: 0.6861 - val_accuracy: 0.7500
Epoch 174/1500
1/1 [==============================] - 0s 244ms/step - loss: 1.3267 - accuracy: 0.5000 - val_loss: 0.6937 - val_accuracy: 0.7500
Epoch 175/1500
1/1 [==============================] - 0s 242ms/step - loss: 0.4119 - accuracy: 0.8750 - val_loss: 0.6994 - val_accuracy: 0.7500
Epoch 176/1500
1/1 [==============================] - 0s 242ms/step - loss: 1.0830 - accuracy: 0.6250 - val_loss: 0.7106 - val_accuracy:

Epoch 227/1500
1/1 [==============================] - 0s 239ms/step - loss: 1.1178 - accuracy: 0.6250 - val_loss: 0.3055 - val_accuracy: 0.8750
Epoch 228/1500
1/1 [==============================] - 0s 241ms/step - loss: 1.2566 - accuracy: 0.2500 - val_loss: 0.3024 - val_accuracy: 0.8750
Epoch 229/1500
1/1 [==============================] - 0s 238ms/step - loss: 1.4338 - accuracy: 0.6250 - val_loss: 0.2928 - val_accuracy: 0.8750
Epoch 230/1500
1/1 [==============================] - 0s 239ms/step - loss: 2.0674 - accuracy: 0.2500 - val_loss: 0.2850 - val_accuracy: 0.8750
Epoch 231/1500
1/1 [==============================] - 0s 245ms/step - loss: 2.0687 - accuracy: 0.5000 - val_loss: 0.2725 - val_accuracy: 0.8750
Epoch 232/1500
1/1 [==============================] - 0s 242ms/step - loss: 2.9752 - accuracy: 0.3750 - val_loss: 0.2710 - val_accuracy: 0.8750
Epoch 233/1500
1/1 [==============================] - 0s 244ms/step - loss: 2.5890 - accuracy: 0.3750 - val_loss: 0.2663 - val_accuracy:

...

1/1 [==============================] - 0s 239ms/step - loss: 1.5640 - accuracy: 0.5000 - val_loss: 0.1176 - val_accuracy: 0.8750
Epoch 1250/1500
1/1 [==============================] - 0s 243ms/step - loss: 0.8473 - accuracy: 0.7500 - val_loss: 0.1211 - val_accuracy: 0.8750
Epoch 1251/1500
1/1 [==============================] - 0s 236ms/step - loss: 2.0850 - accuracy: 0.6250 - val_loss: 0.1234 - val_accuracy: 0.8750
Epoch 1252/1500
1/1 [==============================] - 0s 240ms/step - loss: 0.4402 - accuracy: 0.7500 - val_loss: 0.1273 - val_accuracy: 0.8750
Epoch 1253/1500
1/1 [==============================] - 0s 240ms/step - loss: 2.1814 - accuracy: 0.5000 - val_loss: 0.1424 - val_accuracy: 0.8750
Epoch 1254/1500
1/1 [==============================] - 0s 238ms/step - loss: 0.2690 - accuracy: 0.7500 - val_loss: 0.1421 - val_accuracy: 0.8750
Epoch 1255/1500
1/1 [==============================] - 0s 242ms/step - loss: 0.5059 - accuracy: 0.7500 - val_loss: 0.1399 - val_accuracy: 0.8750
E

1/1 [==============================] - 0s 240ms/step - loss: 1.4343 - accuracy: 0.7500 - val_loss: 0.0392 - val_accuracy: 1.0000
Epoch 1306/1500
1/1 [==============================] - 0s 239ms/step - loss: 1.5354 - accuracy: 0.6250 - val_loss: 0.0422 - val_accuracy: 1.0000
Epoch 1307/1500
1/1 [==============================] - 0s 242ms/step - loss: 0.0439 - accuracy: 1.0000 - val_loss: 0.0437 - val_accuracy: 1.0000
Epoch 1308/1500
1/1 [==============================] - 0s 243ms/step - loss: 1.5069 - accuracy: 0.6250 - val_loss: 0.0424 - val_accuracy: 1.0000
Epoch 1309/1500
1/1 [==============================] - 0s 242ms/step - loss: 0.9731 - accuracy: 0.6250 - val_loss: 0.0393 - val_accuracy: 1.0000
Epoch 1310/1500
1/1 [==============================] - 0s 241ms/step - loss: 1.7030 - accuracy: 0.5000 - val_loss: 0.0376 - val_accuracy: 1.0000
Epoch 1311/1500
1/1 [==============================] - 0s 245ms/step - loss: 0.9335 - accuracy: 0.8750 - val_loss: 0.0390 - val_accuracy: 1.0000
E

1/1 [==============================] - 0s 237ms/step - loss: 0.3445 - accuracy: 0.8750 - val_loss: 0.1499 - val_accuracy: 0.8750
Epoch 1362/1500
1/1 [==============================] - 0s 237ms/step - loss: 0.4603 - accuracy: 0.7500 - val_loss: 0.1424 - val_accuracy: 0.8750
Epoch 1363/1500
1/1 [==============================] - 0s 241ms/step - loss: 0.5111 - accuracy: 0.8750 - val_loss: 0.1342 - val_accuracy: 0.8750
Epoch 1364/1500
1/1 [==============================] - 0s 241ms/step - loss: 0.6111 - accuracy: 0.6250 - val_loss: 0.1266 - val_accuracy: 0.8750
Epoch 1365/1500
1/1 [==============================] - 0s 238ms/step - loss: 0.8156 - accuracy: 0.6250 - val_loss: 0.1147 - val_accuracy: 0.8750
Epoch 1366/1500
1/1 [==============================] - 0s 238ms/step - loss: 1.3822 - accuracy: 0.5000 - val_loss: 0.1098 - val_accuracy: 0.8750
Epoch 1367/1500
1/1 [==============================] - 0s 239ms/step - loss: 0.9025 - accuracy: 0.8750 - val_loss: 0.1007 - val_accuracy: 1.0000
E

1/1 [==============================] - 0s 242ms/step - loss: 0.8367 - accuracy: 0.6250 - val_loss: 0.0316 - val_accuracy: 1.0000
Epoch 1418/1500
1/1 [==============================] - 0s 242ms/step - loss: 0.9440 - accuracy: 0.8750 - val_loss: 0.0319 - val_accuracy: 1.0000
Epoch 1419/1500
1/1 [==============================] - 0s 242ms/step - loss: 0.1449 - accuracy: 1.0000 - val_loss: 0.0340 - val_accuracy: 1.0000
Epoch 1420/1500
1/1 [==============================] - 0s 238ms/step - loss: 2.2206 - accuracy: 0.6250 - val_loss: 0.0336 - val_accuracy: 1.0000
Epoch 1421/1500
1/1 [==============================] - 0s 237ms/step - loss: 0.8687 - accuracy: 0.6250 - val_loss: 0.0324 - val_accuracy: 1.0000
Epoch 1422/1500
1/1 [==============================] - 0s 240ms/step - loss: 1.4011 - accuracy: 0.6250 - val_loss: 0.0321 - val_accuracy: 1.0000
Epoch 1423/1500
1/1 [==============================] - 0s 242ms/step - loss: 0.4868 - accuracy: 0.7500 - val_loss: 0.0328 - val_accuracy: 1.0000
E

1/1 [==============================] - 0s 240ms/step - loss: 1.1189 - accuracy: 0.7500 - val_loss: 0.1283 - val_accuracy: 0.8750
Epoch 1474/1500
1/1 [==============================] - 0s 236ms/step - loss: 0.1466 - accuracy: 1.0000 - val_loss: 0.1282 - val_accuracy: 0.8750
Epoch 1475/1500
1/1 [==============================] - 0s 240ms/step - loss: 0.1826 - accuracy: 0.8750 - val_loss: 0.1287 - val_accuracy: 0.8750
Epoch 1476/1500
1/1 [==============================] - 0s 241ms/step - loss: 0.4182 - accuracy: 0.6250 - val_loss: 0.1316 - val_accuracy: 0.8750
Epoch 1477/1500
1/1 [==============================] - 0s 239ms/step - loss: 0.4573 - accuracy: 0.8750 - val_loss: 0.1409 - val_accuracy: 0.8750
Epoch 1478/1500
1/1 [==============================] - 0s 236ms/step - loss: 0.0811 - accuracy: 1.0000 - val_loss: 0.1441 - val_accuracy: 0.8750
Epoch 1479/1500
1/1 [==============================] - 0s 243ms/step - loss: 1.0839 - accuracy: 0.7500 - val_loss: 0.1522 - val_accuracy: 0.8750
E

In [14]:
import matplotlib.pyplot as plt
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy"])
plt.show()

In [15]:
import matplotlib.pyplot as plt
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("Epoch")
plt.legend(["loss","Validation Loss"])
plt.show()

In [16]:
json_string = net_final.to_json()
with open("resnet_nose_model.config", "w") as text_file:    
    text_file.write(json_string)

In [17]:
#net_final.save(WEIGHTS_FINAL)
net_final.save_weights(WEIGHTS_FINAL)